In [27]:
import sys
import dateutil.parser
import time
import os
import pkg_resources
from multiprocessing import Pool
#pkg_resources.require("boto==2.44.0")
import boto as boto
import boto.ec2 as ec2
import datetime, time
import pandas as pd
import matplotlib.pyplot as plt

#MAKE SURE BOTO VERSION is 2.44.0, Conda seems to only have 2.43 so you must manually install
print (boto.Version)
print(os.path.dirname(boto.__file__))

%pylab inline
%matplotlib inline


instance_types  = [ 't1.micro', 't2.nano', 't2.micro', 't2.small', 't2.medium', 't2.large', 't2.xlarge', 't2.2xlarge', 
                   'm1.small', 'm1.medium', 'm1.large', 'm1.xlarge', 'm3.medium', 'm3.large', 'm3.xlarge', 'm3.2xlarge', 
                   'm4.large', 'm4.xlarge', 'm4.2xlarge', 'm4.4xlarge', 'm4.10xlarge', 'm4.16xlarge', 'm2.xlarge', 
                   'm2.2xlarge', 'm2.4xlarge', 'cr1.8xlarge', 'r3.large', 'r3.xlarge', 'r3.2xlarge', 'r3.4xlarge',
                   'r3.8xlarge', 'r4.large', 'r4.xlarge', 'r4.2xlarge', 'r4.4xlarge', 'r4.8xlarge', 'r4.16xlarge',
                   'x1.16xlarge', 'x1.32xlarge', 'i2.xlarge', 'i2.2xlarge', 'i2.4xlarge', 'i2.8xlarge', 'hi1.4xlarge',
                   'hs1.8xlarge', 'c1.medium', 'c1.xlarge', 'c3.large', 'c3.xlarge', 'c3.2xlarge', 'c3.4xlarge', 
                   'c3.8xlarge', 'c4.large', 'c4.xlarge', 'c4.2xlarge', 'c4.4xlarge', 'c4.8xlarge', 'cc1.4xlarge',
                   'cc2.8xlarge', 'g2.2xlarge', 'g2.8xlarge', 'cg1.4xlarge', 'p2.xlarge', 'p2.8xlarge', 'p2.16xlarge',
                   'd2.xlarge', 'd2.2xlarge', 'd2.4xlarge', 'd2.8xlarge', 'f1.2xlarge', 'f1.16xlarge']
 
#regions = ['us-east-1', 'us-west-2', 'us-west-1', 'eu-west-1','eu-central-1', 'ap-southeast-1', 'ap-northeast-1',
#           'ap-northeast-2', 'ap-southeast-2', 'sa-east-1']

regions = ['us-east-1', 'us-west-2', 'eu-west-1']

number_of_days = 30

#end = !date -u "+%Y-%m-%dT%H:%M:%S"
#end = end[0]
#start = !date -v30d -u "+%Y-%m-%dT%H:%M:%S"
#start = start[0]

#end = datetime.datetime.now()
#start = datetime.datetime.now() + datetime.timedelta(-30) 
end = "2016-12-12T10:46:26+00:00"
start = "2016-10-12T10:46:26+00:00"
print ("will process from " + str(start) + " to " + str(end))
print(len(instance_types))

2.44.0
/Users/ksparakis/Dropbox/Boston Univ/Graduate/CS505 - Datas science/spot-analysis/boto
Populating the interactive namespace from numpy and matplotlib
will process from 2016-10-12T10:46:26+00:00 to 2016-12-12T10:46:26+00:00
71


In [22]:
def indataCreator(regions, instance_types):
    indata=[]
    
    for region in regions:
        count =0
        for instance in instance_types:
            indata.append({'region':region, 'instance':instance, 'count':count})
            count = count+1
            
    return indata

In [25]:
def parralelGrabber(indata):
    region = indata['region']
    instance = indata['instance']
    counter = str(indata['count'])
    
    ec2 = boto.ec2.connect_to_region(region)
    print(str(datetime.datetime.now())+" === "+ counter+"/71 ==== processing "+ instance + " In "+ region + " ***")

    l = []
    prices = ec2.get_spot_price_history(start_time=start, end_time=end, instance_type=instance)
    for price in prices:
        d = {'InstanceType': price.instance_type,                  
             'AvailabilityZone': price.availability_zone, 
             'SpotPrice': price.price,
             'Timestamp': price.timestamp}
        l.append(d)
    next = prices.next_token
    while (next != ''):
        prices = ec2.get_spot_price_history(start_time=start, end_time=end, instance_type=instance, next_token=next)
        for price in prices:
            d = {'InstanceType': price.instance_type, 
                 'AvailabilityZone': price.availability_zone,
                 'SpotPrice': price.price, 
                 'Timestamp': price.timestamp}
            l.append(d)
            next = prices.next_token
    df = pd.DataFrame(l)
    df.to_csv('/Volumes/Lexumo/data/reg/awsData-'+region+'-'+instance+'.csv')

    print(str(datetime.datetime.now())+"==== "+ counter+"/71 ==== FINISHED "+ instance + " In "+ region + " ====")

In [26]:

startTIMER = time.time()
pool = Pool(processes=10)
iner = indataCreator(regions, instance_types)
pool.map(parralelGrabber,iner)
print("==%%%%%=====^^^^^^====timeElapsed:")
print(time.time() - startTIMER)

2016-12-12 14:33:00.499803 === 42/71 ==== processing i2.8xlarge In us-east-1 ***
2016-12-12 14:33:00.482431 === 12/71 ==== processing m3.medium In us-east-1 ***
2016-12-12 14:33:00.484784 === 0/71 ==== processing t1.micro In us-east-1 ***
2016-12-12 14:33:00.488352 === 18/71 ==== processing m4.2xlarge In us-east-1 ***
2016-12-12 14:33:00.489798 === 24/71 ==== processing m2.4xlarge In us-east-1 ***
2016-12-12 14:33:00.485664 === 30/71 ==== processing r3.8xlarge In us-east-1 ***
2016-12-12 14:33:00.489196 === 6/71 ==== processing t2.xlarge In us-east-1 ***
2016-12-12 14:33:00.501752 === 54/71 ==== processing c4.2xlarge In us-east-1 ***
2016-12-12 14:33:00.494098 === 36/71 ==== processing r4.16xlarge In us-east-1 ***
2016-12-12 14:33:00.500011 === 48/71 ==== processing c3.xlarge In us-east-1 ***
2016-12-12 14:33:06.565162==== 6/71 ==== FINISHED t2.xlarge In us-east-1 ====
2016-12-12 14:33:06.565358==== 36/71 ==== FINISHED r4.16xlarge In us-east-1 ====
2016-12-12 14:33:06.576768 === 7/71 =

BotoServerError: BotoServerError: 503 Service Unavailable
<?xml version="1.0" encoding="UTF-8"?>
<Response><Errors><Error><Code>RequestLimitExceeded</Code><Message>Request limit exceeded.</Message></Error></Errors><RequestID>06bcdf02-38ab-4413-a036-4a417518a7b7</RequestID></Response>

2016-12-12 15:08:08.610343==== 24/71 ==== FINISHED m2.4xlarge In us-west-2 ====
2016-12-12 15:08:08.959678 === 2/71 ==== processing t2.micro In eu-west-1 ***
2016-12-12 15:08:09.755660==== 2/71 ==== FINISHED t2.micro In eu-west-1 ====
2016-12-12 15:08:09.761700 === 3/71 ==== processing t2.small In eu-west-1 ***
2016-12-12 15:08:10.370111==== 3/71 ==== FINISHED t2.small In eu-west-1 ====
2016-12-12 15:08:10.376589 === 4/71 ==== processing t2.medium In eu-west-1 ***
2016-12-12 15:08:10.992894==== 4/71 ==== FINISHED t2.medium In eu-west-1 ====
2016-12-12 15:08:10.999661 === 5/71 ==== processing t2.large In eu-west-1 ***
2016-12-12 15:08:11.646758==== 5/71 ==== FINISHED t2.large In eu-west-1 ====
2016-12-12 15:08:11.652545 === 6/71 ==== processing t2.xlarge In eu-west-1 ***
2016-12-12 15:08:12.399487==== 6/71 ==== FINISHED t2.xlarge In eu-west-1 ====
2016-12-12 15:08:12.405757 === 7/71 ==== processing t2.2xlarge In eu-west-1 ***
2016-12-12 15:08:13.010344==== 7/71 ==== FINISHED t2.2xlarge 

In [28]:
parralelGrabber({'region':'us-west-2', 'instance':'m3.2xlarge', 'count':15})

2016-12-12 16:24:33.129893 === 15/71 ==== processing m3.2xlarge In us-west-2 ***
2016-12-12 16:32:35.883149==== 15/71 ==== FINISHED m3.2xlarge In us-west-2 ====


In [2]:
def regGrabber(region):  
    ec2 = boto.ec2.connect_to_region(region)
    print(str(datetime.datetime.now())+" === processing " + region + " ===")

    for instance in instance_types:
        l = []
        sys.stdout.write(str(datetime.datetime.now())+" *** processing " + instance + " In "+ region + " ***\n")
        sys.stdout.flush()
        prices = ec2.get_spot_price_history(start_time=start, end_time=end, instance_type=instance)
        for price in prices:
            d = {'InstanceType': price.instance_type, 
                 'AvailabilityZone': price.availability_zone, 
                 'SpotPrice': price.price, 
                 'Timestamp': price.timestamp}
            l.append(d)
        next = prices.next_token
        while (next != ''):
            #sys.stdout.write(".")
            #sys.stdout.flush()
            prices = ec2.get_spot_price_history(start_time=start, end_time=end, instance_type=instance, next_token=next)
            for price in prices:
                d = {'InstanceType': price.instance_type, 
                     'AvailabilityZone': price.availability_zone, 
                     'SpotPrice': price.price, 
                     'Timestamp': price.timestamp}
                l.append(d)
            next = prices.next_token
        df = pd.DataFrame(l)
        df.to_csv('/Volumes/Lexumo/data/reg/awsData-'+region+'-'+instance+'.csv')
        #sys.stdout.write("\n")
    print("====FINISHED "+ region +" ====")

In [3]:
startTIMER = time.time()
pool = Pool(processes=8)
pool.map(regGrabber,regions)
print("==%%%%%=====^^^^^^====timeElapsed:")
print(time.time() - startTIMER)
'''
l = []
for region in regions:
    ec2 = boto.ec2.connect_to_region(region)
    print("=== processing " + region + " ===")

    for instance in instance_types:
        sys.stdout.write("*** processing " + instance + " In "+ region + " ***\n")
        sys.stdout.flush()
        prices = ec2.get_spot_price_history(start_time=start, end_time=end, instance_type=instance)
        for price in prices:
            d = {'InstanceType': price.instance_type, 
                 'AvailabilityZone': price.availability_zone, 
                 'SpotPrice': price.price, 
                 'Timestamp': price.timestamp}
            l.append(d)
        next = prices.next_token
        while (next != ''):
            sys.stdout.write(".")
            sys.stdout.flush()
            prices = ec2.get_spot_price_history(start_time=start, end_time=end, instance_type=instance, next_token=next)
            for price in prices:
                d = {'InstanceType': price.instance_type, 
                     'AvailabilityZone': price.availability_zone, 
                     'SpotPrice': price.price, 
                     'Timestamp': price.timestamp}
                l.append(d)
            next = prices.next_token
        sys.stdout.write("\n")

df = pd.DataFrame(l)
df.to_csv('/Volumes/Lexumo/data/awsData.csv')
#df = df.set_index(pd.to_datetime(df['Timestamp']))
'''

2016-12-12 06:22:07.256739 === processing ap-southeast-1 ===
2016-12-12 06:22:07.248631 === processing eu-west-1 ===
2016-12-12 06:22:07.251946 === processing eu-central-1 ===
2016-12-12 06:22:07.248206 === processing us-west-2 ===
2016-12-12 06:22:07.251696 === processing us-west-1 ===
2016-12-12 06:22:07.253785 === processing us-east-1 ===
2016-12-12 06:22:07.261276 *** processing t1.micro In eu-west-1 ***
2016-12-12 06:22:07.256487 === processing ap-northeast-1 ===
2016-12-12 06:22:07.261691 *** processing t1.micro In ap-southeast-1 ***
2016-12-12 06:22:07.264373 *** processing t1.micro In eu-central-1 ***
2016-12-12 06:22:07.264360 === processing ap-northeast-2 ===
2016-12-12 06:22:07.277237 *** processing t1.micro In us-west-2 ***
2016-12-12 06:22:07.280792 *** processing t1.micro In us-west-1 ***
2016-12-12 06:22:07.285405 *** processing t1.micro In us-east-1 ***
2016-12-12 06:22:07.291050 *** processing t1.micro In ap-northeast-1 ***
2016-12-12 06:22:07.306290 *** processing t1.

EC2ResponseError: EC2ResponseError: 401 Unauthorized
<?xml version="1.0" encoding="UTF-8"?>
<Response><Errors><Error><Code>AuthFailure</Code><Message>AWS was not able to validate the provided access credentials</Message></Error></Errors><RequestID>ef9041e7-5ce3-4f73-812d-48248d0a2604</RequestID></Response>


2016-12-12 10:11:45.456170 *** processing r4.large In eu-west-1 ***

2016-12-12 10:11:49.513543 *** processing r4.xlarge In eu-west-1 ***

2016-12-12 10:11:52.760172 *** processing r4.2xlarge In eu-west-1 ***

2016-12-12 10:12:06.570888 *** processing r4.4xlarge In eu-west-1 ***

2016-12-12 10:12:09.008298 *** processing r4.8xlarge In eu-west-1 ***

2016-12-12 10:12:11.056906 *** processing r4.16xlarge In eu-west-1 ***

2016-12-12 10:12:12.422706 *** processing x1.16xlarge In eu-west-1 ***

2016-12-12 10:12:28.750139 *** processing x1.32xlarge In eu-west-1 ***

2016-12-12 10:12:58.244848 *** processing i2.xlarge In eu-west-1 ***

2016-12-12 10:17:25.163710 *** processing i2.2xlarge In eu-west-1 ***

2016-12-12 10:19:01.557935 *** processing m4.16xlarge In us-west-2 ***

2016-12-12 10:19:10.446073 *** processing c4.2xlarge In ap-southeast-1 ***

2016-12-12 10:19:14.319380 *** processing i2.4xlarge In eu-west-1 ***

2016-12-12 10:22:35.710461 *** processing m2.xlarge In us-west-2 ***

2